# Google Scholar 
Try avoid robots by explicit clicking on the previously loaded 
page

See: https://academia.stackexchange.com/q/2567

Check for https://github.com/ckreibich/scholar.py/blob/master/scholar.py

https://pypi.org/project/googletrans/

See also: lookup fields:
https://scholar.google.co.in/scholar_lookup?title=Estimates+for+the+number+of+sums+and+products+and+for+exponential+sums+in+fields+of+prime+order&author=Jean+Bourgain&author=AA+Glibichuk&author=SERGEI+VLADIMIROVICH+Konyagin&year=2006&doi=10.1112/S0024610706022721&publisher=Oxford+University+Press&journal=Journal+of+the+London+Mathematical+Society&volume=73&issue=2&pages=380-398&hl=en

https://portal.ictp.it/icts/howto/proxy.html

## Choose between selenium or requests

In [1]:
url='https://scholar.google.com'
CITES=False
if CITES:
    url=url+'/scholar?cites=16729892078990709043&as_sdt=2005&sciodt=0,5&hl=es'
SELENIUM=False
if SELENIUM:
    #pip3 install webdriver_manager
    from webdriver_manager.firefox import GeckoDriverManager
    from selenium import webdriver
    from pathlib import Path
    home = str(Path.home())
    try:
        browser = webdriver.Firefox(executable_path=
          '/home/restrepo/.wdm/geckodriver/v0.23.0/linux64/geckodriver')
    except:
        browser = webdriver.Firefox(
            executable_path= GeckoDriverManager().install())
    
    browser.get(url)
    #TODO: Authentication:
    # Login page
    # gs_hdr_act_s by id

In [ ]:
browser.find_element_by_id("gs_hdr_act_s").click()

browser.find_element_by_id("identifierId").clear()

browser.find_element_by_id("identifierId").send_keys("admin@fisica.udea.edu.co")

browser.find_element_by_class_name("RveJvd").click()

In [ ]:
import getpass

password=getpass.getpass()

In [4]:
browser.find_element_by_class_name("whsOnd").send_keys(password)

browser.find_element_by_class_name("RveJvd").click()

In [2]:
if not SELENIUM:
    import requests
    headers_Get = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }
    r=requests.Session()
    browser=None

In [22]:
#import googlescholar as gs
#%%writefile ../cienciometria/googlescholar.py
import Levenshtein
import re
from bs4 import BeautifulSoup
import requests
import scholarly

def scholarly_to_gs(d):
    if not d:
        return d
    gsr={}
    try:
        gsr['cites']=d.citedby
        gsr['cites_link']=d.id_scholarcitedby
    except AttributeError:
        gsr['cites']=0
        gsr['cites_link']=''
    gsr['title']=d.bib.get('title')
    gsr['ref']=d.bib.get('author').replace(' and',',')
    gsr['authors']=gsr['ref'].split('-')[0].strip()
    gsr['PDF']=d.bib.get('eprint')
    
    try:
        jy=gsr['ref'].split('-')[1].strip()
        gsr['Journal']=jy.split(',')[0]
        try:
            gsr['Year']=eval(jy.split(',')[1])
        except:
            gsr['Year']=-1
    except:
        gsr['Journal']=''

    gsr['abstract']=d.bib.get('abstract')
    return gsr

def firefox_get(url,browser=browser):
    try:
        if browser:
            if url.find('scholar_lookup?')==-1:
                q=url.split('=')[-1]
                browser.find_element_by_id("gs_hdr_tsi").clear()
                browser.find_element_by_id("gs_hdr_tsi").send_keys(q)
                browser.find_element_by_id('gs_hdr_tsb').click()
            else:
                browser.get(url)
        else:
            rget=r.get(url,headers=headers_Get)
    except:
        return 'FAILED'
    
    if browser:
        html=browser.page_source
    else:
        html = rget.text
    if html.lower().find('gs_captcha_f')>-1:
        if browser:
            input('check robots')
        else:
            sys.exit('ERROR: Captcha anti-robots requested!\n Aborting execution.')
    #if html.find('Sorry')>-1:
    #    input('check sorry')

    return html

def get_google_scholar(record):
    '''
    Analyise the BeautifulSoup record for an article 
    in Google Scholar.
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','Jornal','Year',
    'abstract','cites','cites_link'
    '''
    import random
    import time
    gsr={}
    try:
        cites=record.find_all('a',{"href":re.compile( "/scholar\?cites=" )})[0]
        try:
            gsr['cites']=int( cites.text.split()[-1] )
            gsr['cites_link']=cites.attrs.get('href')
        except:
            gsr['cites']=0
    except:
        cites=None

    # Title
    try:
        #The .split('XXX')[-1]  does not afect the result when .text does no contains 'XXX'
        tc=record.find_all('h3',{"class":"gs_rt"})[0].text.split('[CITATION][C] ')[-1]
    except:
        tc=''

    gsr['title']=tc.strip().split('[HTML][HTML] ')[-1].split(
                                  '[PDF][PDF] '  )[-1]
    
    # Explore authors, google scholar profile - Journal, Year - Publisher
    gpa=None
    try:
        gpa=record.find_all('div',{"class":"gs_a"})[0]
        #Full ref with authors, google scholar profile - Journal, Year - Publisher
        ref=gpa.text
        gsr['ref']=ref.strip()
        refparts=ref.split('\xa0-')
    except IndexError:
        gsr['ref']=''
        refparts=[]

    try:
        gsr['authors']=refparts[0]
    except IndexError:    
        gsr['authors']=''
        
    try:
        journalparts=refparts[-1].strip().split(' - ')
        gsr['publisher']=journalparts[-1]
        gsr['Journal']=journalparts[0].split(',')[0]
        gsr['Year']=journalparts[0].split(',')[-1].strip()
    except IndexError:
        gsr['publisher']=''
        gsr['Journal']=''
        gsr['Year']

    #Abstract:
    try:
        gsr['abstract']=record.find_all('div',{'class':'gs_rs'})[0].text.replace('\xa0…','')
    except:
        gsr['abstract']=''
    # citations
    if gpa:
        lpr=gpa.find_all("a",{ "href":re.compile("/citations\?user=*")   } )
        prf={}
        for pr in lpr:
            prf[ pr.text ]=pr.attrs.get('href').split('?')[-1].split('&')[0].split('user=')[-1]
        gsr['profiles']=prf
    
    time.sleep( random.randint(1,3)  ) # avoid robots
    return gsr

def request_google_scholar_url(url):
    return requests.get(url)

def google_scholar_page(html):
    '''
    Convert a Google Scholar page into a list
    of dictionaries with metadata info
    '''
    if html.lower().find('gs_captcha_f')>-1:
        input('check robots')
   
    soup = BeautifulSoup(html, "html.parser")
    rgs=soup.find_all('div', {'class':'gs_ri' })

    citations=[]
    for record in rgs:
        citations.append( get_google_scholar(record) )
        
    return citations

def google_scholar_query(title='', author='', coauthors=[], DOI='', year=0, publisher='',
                         journal='', volume='', issue='', pages=0,
                         DEBUG=False,Scholarly=False):
    '''
    Search Google scholar with `sholar_lookup` full fields.
    Only the first result is analized. The output includes 
    a quality measurements between the query and the results 
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','cites','cites_link',
    'quality_title','quality_author'
    '''
    gs={}
    # + → %2B in query formula:
    baseurl='https://scholar.google.com/'
    q='scholar_lookup?'
    
    nl=0
    if title:
        nl=nl+1
        q= q+'title={}'.format(title.replace(' ','+'))
    if author:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'author={}'.format(author.replace(' ','+'))
    if DOI:
        if nl: q= q+'&'
        nl=nl+1    
        q= q+'doi={}'.format(DOI)
    if year:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'year={}'.format(year)
    if publisher:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'publisher={}'.format(publisher.replace(' ','+'))
    if journal:
        if nl: q= q+'&'
        nl=nl+1     
        q= q+'journal={}'.format(journal.replace(' ','+'))
    if volume:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'volume={}'.format(volume)
    if issue:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'issue={}'.format(issue)   
    if pages:
        if nl: q= q+'&'
        nl=nl+1
        q= q+'pages={}'.format(pages)
    if coauthors:
        for i in coauthors :
            if nl: q= q+'&'
            nl=nl+1
            q= q+'author={}'.format(i.replace(' ','+'))    
    url=baseurl+q
    if DEBUG:
        print(url)   
    
    #s = requests.Session()
    if not Scholarly:
        rtext=firefox_get(url)

        #soup = BeautifulSoup(r.text, "html.parser")
        soup = BeautifulSoup(rtext, "html.parser")


        # Main contents:
        rgs=soup.find_all('div', {'class':'gs_ri' })

        try:
            record=rgs[0]
        except IndexError:
            #exit if record not found and returns empty dictionary
            return gs
    
        gs.update(get_google_scholar(record))
    else:
        search_query = scholarly.search_pubs_query(q)

        try:
            d=next(search_query)
        except StopIteration:
            return {}     
        gs=scholarly_to_gs(d)
    
    #Check if author is in authors list
    if gs and author:
        sau=0
        for a in gs['authors'].split(','):
            saun=Levenshtein.ratio(author.lower(),a.lower().strip())
            if saun>sau:
                sau=saun
                
        gs['quality_author']=round(sau,2)
    else:
        gs['quality_author']=-1 #-1 means not checked
        
    if gs and title:
        gs['quality_title']=round( Levenshtein.ratio(
                   title.lower(),gs['title'].lower() ),2 )
    else:
        gs['quality_title'] =-1 #-1 means not checked
        
    #EXTRA FIELDS:
    #PDF
    if not Scholarly:
        try:
            gs['PDF']=soup.find_all('div',
                        {"class":"gs_or_ggsm"})[0].find_all('a')[0].get("href")
        except:
            gs['PDF']=''

    if DEBUG:
        print('='*80)
        print(record)
        
        return gs,record
    else:
        return gs

def get_cites_refs(browser,url,maxcites=65,t=60):
    import random
    import time
    url='https://scholar.google.com'+url
    browser.get(url)
     
    endpage=int(maxcites/10)+1
    refs=''
    
    kk=google_scholar_page( browser.page_source )
    try:
        refs=refs+'\n'.join( list((pd.DataFrame(kk)['title']+'; '
                                  +pd.DataFrame(kk)['ref']).values) )+'\n' 
    except:
        refs=''
    
    
    for i in range(endpage):
        try:
            browser.find_element_by_class_name('gs_ico_nav_next').click()
            kk=google_scholar_page( browser.page_source )
            try:
                refs=refs+'\n'.join( list( (pd.DataFrame(kk)['title']+'; '
                                +pd.DataFrame(kk)['ref']).values ) )+'\n' 
            except:
                refs=''
        except:
            break
            
    time.sleep(random.uniform(0.9*t,1.1*t))
    return refs

In [5]:
import wosplus as wp

In [6]:
%%writefile drive.cfg
[FILES]
DOIS.xlsx               = 1bikNT7Gmp4G7dfeMuGsF-az7D8lskK0O
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x
oaudea.json             = 1BmqRoQDgfpOFjXBVEfI999uaN6XX6Fkd
datos1.csv              = 11CyLRZZwVbgw6YAC-igRJ3mrkIwk0aaiXnd-EOofYTI
RedalycMetadatosArticulos.csv= #See Redalyc_GoogleScholar_run

Overwriting drive.cfg


In [7]:
oa=wp.wosplus('drive.cfg')

In [8]:
oaudea=oa.read_drive_csv('RedalycMetadatosArticulos.csv')

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [9]:
oaudea=wp.fill_NaN(oaudea)

In [10]:
oaudea.shape

(629851, 13)

In [11]:
datos=oaudea.copy()

In [25]:
#JUST START AFTER FINISH REDALYC
nini=0# initial doi
n=3000 # Total of DOIs
nend=nini+n
T=12 #hours of search
t=T/n*3600 # [s] query time
t=60
day=24*3600 #s
mintime=0.9*t*n # [s] minimal time search
wait=30#day-mintime # maximum wait

In [13]:
t=60

In [14]:
datos=datos[datos.DOI!=''].reset_index(drop=True)

In [15]:
datos.columns

Index(['REVISTA', 'INSTITUCION_REVISTA', 'PAIS_REVISTA', 'IDARTICULO',
       'TITULO', 'Autor(es)', 'DOI', 'ANIO', 'VOLUMEN', 'NUMERO', 'PAGINAS',
       'IDIOMA', 'RESUMEN'],
      dtype='object')

In [16]:
ii=1
(datos.loc[ii,'DOI'],
 datos.loc[ii,'TITULO'],
 datos.loc[ii,'Autor(es)'].split(', ')[0],
 datos.loc[ii,'Autor(es)'].split('\n')[1:4],
 datos.loc[ii,'ANIO'],
 datos.loc[ii,'REVISTA'],
)

('10.4067/S0717-554X2017000100013',
 'Ventajas del análisis sistémico aplicado a los espacios locales',
 'Manuela Ortega ,M. Concepción Segovia ',
 [],
 2017,
 'Cinta de Moebio')

In [17]:
datos.shape

(18815, 13)

In [18]:
datos[nini:n].index

RangeIndex(start=0, stop=3000, step=1)

In [19]:
datos.loc[ii,'ANIO']

2017

In [23]:
import pandas as pd
import time
import random
Scholarly=False
doi='DOI'
title_simple='TITULO'
article_id='IDARTICULO'

d=google_scholar_query(title=datos.loc[ii,title_simple],
                    author=datos.loc[ii,'Autor(es)'].split(', ')[0],
                    coauthors=datos.loc[ii,'Autor(es)'].split(', ')[1:4],
                    DOI=datos.loc[ii,doi],
                    year=datos.loc[ii,'ANIO'],
                    journal=datos.loc[ii,'REVISTA'],Scholarly=Scholarly)
d

{'Journal': 'Cinta de moebio',
 'PDF': 'https://scielo.conicyt.cl/scielo.php?pid=S0717-554X2017000100013&script=sci_arttext',
 'Year': '2017',
 'abstract': 'This paper shows how system analysis applied to local spaces facilitate the generation of proposals for integral development in the field of sustainability. Regions are composed of physical and social components generating products that benefit or harm themselves. These resources can be exploited or remained idle for lack of actors to develop them. They react to internal and external actions, both human and natural. Among these actions are the policies for local development. For these policies to be efficient, they need comprehensive studies',
 'authors': 'M Ortega, MC Segovia',
 'cites': 1,
 'cites_link': '/scholar?cites=14403974261697570786&as_sdt=2005&sciodt=0,5&hl=en',
 'profiles': {'M Ortega': 'GkYFaLMAAAAJ'},
 'publisher': 'scielo.conicyt.cl',
 'quality_author': 0.42,
 'quality_title': 1.0,
 'ref': 'M Ortega, MC Segovia\xa0- 

In [ ]:
import pandas as pd
import time
import random
Scholarly=False
doi='DOI'
title_simple='TITULO'
article_id='IDARTICULO'
dfgs=pd.DataFrame()
ibrkn=0
maxibrn=400

for ii in datos[nini:n].index:
    print(ii,datos.loc[ii,doi])
    #gsd=google_scholar_query(DOI=doi)
    gsd=google_scholar_query(title=datos.loc[ii,title_simple],
                    author=datos.loc[ii,'Autor(es)'].split(', ')[0],
                    coauthors=datos.loc[ii,'Autor(es)'].split(', ')[1:4],
                    DOI=datos.loc[ii,doi],
                    year=datos.loc[ii,'ANIO'],
                    journal=datos.loc[ii,'REVISTA'],Scholarly=Scholarly )
            
    gsd['old_title']=datos.loc[ii,'TITULO']
    gsd['DOI']=datos.loc[ii,'DOI']
    gsd['ID_ARTICLE']=datos.loc[ii,article_id]
    dfgs=dfgs.append(gsd,ignore_index=True )
    dfgs.to_json('rdlycdoi.json')
    time.sleep(random.uniform(0.9*t,1.1*t))